# Random Linear Code Encription (RLCE) Python Implementation



Notes:
- It demonstrates the RLCE
  matrix pipeline (S, G1, A, P) and decryption algebraically.


In [12]:
import numpy as np
import galois   # pip install galois
from typing import Tuple

In [13]:
# ---------- Gauss-Jordan inverse that works on galois.FieldArray ----------
def gf_matrix_inverse(A: "galois.FieldArray") -> "galois.FieldArray":
    """
    Invert a square matrix A over a galois FieldArray using Gauss-Jordan.
    Works because FieldArray supports +,-,*,/, comparisons, and row ops.
    """
    GF = type(A)  # A is a FieldArray class instance; GF constructor is its class
    n = A.shape[0]
    # Build augmented matrix [A | I]
    Aug = np.concatenate([A.copy(), GF(np.eye(n, dtype=int))], axis=1)
    Aug = Aug.copy()
    for col in range(n):
        # find pivot row
        pivot = None
        for r in range(col, n):
            if Aug[r, col] != 0:
                pivot = r
                break
        if pivot is None:
            raise np.linalg.LinAlgError("Singular matrix in GF")
        # swap if needed
        if pivot != col:
            Aug[[col, pivot]] = Aug[[pivot, col]]
        # normalize pivot row
        piv = Aug[col, col]
        Aug[col] = Aug[col] / piv
        # eliminate other rows
        for r in range(n):
            if r != col:
                factor = Aug[r, col]
                Aug[r] = Aug[r] - factor * Aug[col]
    inv = Aug[:, n:]
    return inv



In [14]:
# ---------- RLCE building blocks (GF version) ----------
def generate_systematic_Gs_gf(GF, k: int, n: int):
    """Systematic generator Gs over GF (k x n) with first k columns = I_k"""
    assert n >= k
    I = GF(np.eye(k, dtype=int))
    R = GF.Random((k, n - k))
    return GF(np.concatenate([I, R], axis=1))



In [15]:
def random_invertible_field_matrix(GF, size: int):
    """Random invertible 'size x size' matrix in GF via trying until invertible."""
    while True:
        M = GF.Random((size, size))
        try:
            inv = gf_matrix_inverse(M)
            return M, inv
        except np.linalg.LinAlgError:
            continue



In [16]:
def rlce_keygen(GF, k: int, n: int, r: int):
    """
    Generate RLCE public/private keys over field GF.
    Returns (G_public, private_dict)
    private_dict has keys: S, S_inv, Gs, A_blocks (list of Ai and Ai_inv), perm, r
    """
    Gs = generate_systematic_Gs_gf(GF, k, n)   # k x n
    # build random Ci blocks
    Cs = [GF.Random((k, r)) for _ in range(n)]
    blocks = []
    A_blocks = []
    for i in range(n):
        gi = Gs[:, i].reshape(k, 1)            # k x 1
        Ci = Cs[i]                             # k x r (FieldArray)
        B = GF(np.concatenate([gi, Ci], axis=1))
        Ai, Ai_inv = random_invertible_field_matrix(GF, r + 1)
        A_blocks.append((Ai, Ai_inv))
        Bi = B @ Ai                            # k x (r+1)
        blocks.append(Bi)
    G1A = GF(np.concatenate(blocks, axis=1))  # k x n(r+1)
    # S (k x k) random invertible
    S, S_inv = random_invertible_field_matrix(GF, k)
    G_temp = S @ G1A                          # k x n(r+1)
    # random permutation of columns
    total_cols = G_temp.shape[1]
    perm = np.random.permutation(total_cols)
    # apply permutation (permute columns)
    G = G_temp[:, perm]
    priv = {"S": S, "S_inv": S_inv, "Gs": Gs, "A_blocks": A_blocks, "perm": perm, "r": r}
    return G, priv



In [17]:
def rlce_encrypt(m: "galois.FieldArray", G: "galois.FieldArray", n: int, r: int, t: int) -> "galois.FieldArray":
    """
    Encrypt in GF: m shape (k,), G shape (k, n(r+1)), produce ciphertext y (length n(r+1))
    We build an error vector that sets at most t blocks non-zero (each block length r+1).
    """
    k = m.size
    GF = type(m)
    N = n * (r + 1)
    # choose t distinct block indices
    block_idx = np.random.choice(n, size=t, replace=False)
    e = GF.Zeros(N)
    for i in block_idx:
        # random nonzero block
        block = GF.Random((r+1,))
        # ensure not all zeros (rare)
        if np.all(block == 0):
            block[0] = GF(1)
        e[i*(r+1):(i+1)*(r+1)] = block
    y = (m @ G) + e
    return y



In [18]:
def rlce_decrypt(y: "galois.FieldArray", priv: dict) -> Tuple["galois.FieldArray", bool]:
    """
    Decrypt ciphertext y using private key; returns recovered message and success flag.
    This works because we used systematic Gs — the first k first-block-elements correspond to mS.
    """
    S_inv = priv["S_inv"]; A_blocks = priv["A_blocks"]; perm = priv["perm"]; r = priv["r"]
    n = len(A_blocks)
    GF = type(S_inv)
    # invert permutation
    inv_perm = np.argsort(perm)
    yPinv = y[inv_perm]
    # apply A^{-1} blockwise
    restored_first_elems = []
    for i in range(n):
        block = yPinv[i*(r+1):(i+1)*(r+1)].reshape(1, r+1)   # 1 x (r+1)
        Ai_inv = A_blocks[i][1]                             # Ai_inv
        restored = block @ Ai_inv                          # 1 x (r+1)
        restored_first_elems.append(restored[0, 0])        # first element of each block
    yprime = GF(np.array(restored_first_elems))
    # first k entries are mS (systematic Gs)
    mS = yprime[:S_inv.shape[0]]
    m_recovered = S_inv @ mS
    # Optionally compute check weight(y - mG) etc (omitted).
    return m_recovered, True




In [19]:

np.random.seed(2)
# choose field GF(2^8) for demo (you can pick other p^m)
GF = galois.GF(2**8)   # create a FieldArray subclass for GF(2^8). See docs for GF(...).
k, n = 4, 6
r = 1
t = 1
# keygen
G_pub, priv = rlce_keygen(GF, k, n, r)
# message m as GF array
m = GF([5, 7, 9, 11])   # these are field elements (integers interpreted in GF)
y = rlce_encrypt(m, G_pub, n, r, t)
m_rec, ok = rlce_decrypt(y, priv)
print("original m :", m)
print("recovered m:", m_rec)

original m : [ 5  7  9 11]
recovered m: [15 57 84  3]
